<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [2]:
# !pip3 install beautifulsoup4
# !pip3 install requests

In [3]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [4]:
def date_time(table_cells, text=False):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """

    if not text:
        return [data_time.strip() for data_time in list(table_cells.strings)][0:2]
    else:
        return [data_time.strip() for data_time in list(table_cells)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells, text=False):
    if not text:
        mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    else:
        mass=unicodedata.normalize("NFKD", table_cells).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [5]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [6]:
# use requests.get() method with the provided static_url
# assign the response to a object

data = requests.get(static_url)

Create a `BeautifulSoup` object from the HTML `response`


In [7]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
content = BeautifulSoup(data.content, "lxml")

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [8]:
# Use soup.title attribute
content.title.text

'List of Falcon 9 and Falcon Heavy launches - Wikipedia'

### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [9]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables = content.find_all("table")

Starting from the third table is our target table contains the actual launch records.


In [10]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">1
</th>
<td>

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [11]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
for th in first_launch_table.find_all("th"):
    name = extract_column_from_header(th)
    if name is None:
        continue
    elif len(name) == 0:
        continue
    column_names += [name]

column_names 

['Flight No.',
 'Date and time ( )',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launch outcome']

Check the extracted column names


In [12]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [13]:
def initiate_launch_dict():
    launch_dict= dict.fromkeys(column_names)

    # Remove an irrelvant column
    del launch_dict['Date and time ( )']

    # Let's initial the launch_dict with each value to be an empty list
    launch_dict['Flight No.'] = []
    launch_dict['Launch site'] = []
    launch_dict['Payload'] = []
    launch_dict['Payload mass'] = []
    launch_dict['Orbit'] = []
    launch_dict['Customer'] = []
    launch_dict['Launch outcome'] = []
    # Added some new columns
    launch_dict['Version Booster']=[]
    launch_dict['Booster landing']=[]
    launch_dict['Date']=[]
    launch_dict['Time']=[]

    return launch_dict

launch_dict = initiate_launch_dict()

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [14]:
# extracted_row = 0
# #Extract each table 
# for table_number,table in enumerate(content.find_all('table',"wikitable plainrowheaders collapsible")):
#    # get table row 
#     for rows in table.find_all("tr"):
#         try:
#             a = {}
#             #check to see if first table heading is as number corresponding to launch a number 
#             if rows.th:
#                 if rows.th.string:
#                     flight_number=rows.th.string.strip()
#                     flag=flight_number.isdigit()
#                     a["Flight No."] = flight_number
#             else:
#                 flag=False
#             #get table element 
#             row=rows.find_all('td')
#             #if it is number save cells in a dictonary 
#             if flag:
#                 extracted_row += 1
#                 # Flight Number value
#                 # TODO: Append the flight_number into launch_dict with key `Flight No.`
#                 #print(flight_number)
#                 datatimelist=date_time(row[0])
                
#                 # Date value
#                 # TODO: Append the date into launch_dict with key `Date`
#                 date = datatimelist[0].strip(',')
#                 a["Date"] = date
#                 #print(date)
                
#                 # Time value
#                 # TODO: Append the time into launch_dict with key `Time`
#                 time = datatimelist[1]
#                 a["Time"] = time
#                 #print(time)
                
#                 # Booster version
#                 # TODO: Append the bv into launch_dict with key `Version Booster`
#                 bv=booster_version(row[1])
#                 if not(bv):
#                     bv=row[1].a.string
#                 print(bv)
                
#                 # Launch Site
#                 # TODO: Append the bv into launch_dict with key `Launch Site`
#                 launch_site = row[2].a.string
#                 a["Launch site"] = launch_site
#                 #print(launch_site)
                
#                 # Payload
#                 # TODO: Append the payload into launch_dict with key `Payload`
#                 payload = row[3].a.string
#                 a["Payload"] = payload
#                 #print(payload)
                
#                 # Payload Mass
#                 # TODO: Append the payload_mass into launch_dict with key `Payload mass`
#                 payload_mass = get_mass(row[4])
#                 a["Payload mass"] = payload_mass
#                 #print(payload)
                
#                 # Orbit
#                 # TODO: Append the orbit into launch_dict with key `Orbit`
#                 orbit = row[5].a.string
#                 a["Orbit"] = orbit
#                 #print(orbit)
                
#                 # Customer
#                 # TODO: Append the customer into launch_dict with key `Customer`
#                 customer = row[6].a.string
#                 a["Customer"] = customer
#                 #print(customer)
                
#                 # Launch outcome
#                 # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
#                 launch_outcome = list(row[7].strings)[0]
#                 a["Launch outcome"] = launch_outcome
#                 #print(launch_outcome)
                
#                 # Booster landing
#                 # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
#                 booster_landing = landing_status(row[8])
#                 a["Booster landing"] = booster_landing
#                 #print(booster_landing)
#             for k in launch_dict.keys():
#                 launch_dict[k] += [a[k]]
        
#         except Exception as e:
#             continue

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [15]:
def clean_text(text):
    list_replace = ["\\\n", "\n", "\t", "\xa0"]
    for r in list_replace:
        text = text.replace(r, "")
    
    remove_list = ["[{}]".format(x) for x in range(1000)]
    for r in remove_list:
        if r in text:
            text = text.replace(r, "")
    return text

list_data = []
wikitables = content.find_all("table", "wikitable plainrowheaders collapsible")
flight_no = None
for table in wikitables:
    for r, tr in enumerate(table.find_all("tr")):
        f = tr.find_all("th")
        if len(f) != 0:
            # print([r, f[0].text])
            flight_no = clean_text(f[0].text)
        list_r = []
        for d, td in enumerate(tr.find_all("td")):
            list_r += [clean_text(td.text)]
        if len(list_r) <= 1:
            continue
        list_data += [[r] + list_r + [flight_no]]

list_data

[[1,
  '4 June 2010,18:45',
  'F9 v1.0B0003.1',
  'CCAFS,SLC-40',
  'Dragon Spacecraft Qualification Unit',
  '',
  'LEO',
  'SpaceX',
  'Success',
  'Failure(parachute)',
  '1'],
 [3,
  '8 December 2010,15:43',
  'F9 v1.0B0004.1',
  'CCAFS,SLC-40',
  'Dragon demo flight C1(Dragon C101)',
  '',
  'LEO (ISS)',
  'NASA (COTS)NRO',
  'Success',
  'Failure(parachute)',
  '2'],
 [5,
  '22 May 2012,07:44',
  'F9 v1.0B0005.1',
  'CCAFS,SLC-40',
  'Dragon demo flight C2+(Dragon C102)',
  '525kg (1,157lb)',
  'LEO (ISS)',
  'NASA (COTS)',
  'Success',
  'Not attempted',
  '3'],
 [7,
  '8 October 2012,00:35',
  'F9 v1.0B0006.1',
  'CCAFS,SLC-40',
  'SpaceX CRS-1(Dragon C103)',
  '4,700kg (10,400lb)',
  'LEO (ISS)',
  'NASA (CRS)',
  'Success',
  'No attempt',
  '4'],
 [8, 'Orbcomm-OG2', '172kg (379lb)', 'LEO', 'Orbcomm', 'Partial failure', '4'],
 [10,
  '1 March 2013,15:10',
  'F9 v1.0B0007.1',
  'CCAFS,SLC-40',
  'SpaceX CRS-2(Dragon C104)',
  '4,877kg (10,752lb)',
  'LEO (ISS)',
  'NASA (CRS)'

In [16]:
def split_date_time(text):
    formats = [
        "%d %b %Y%H:%M",
        "%d %b %Y",
        
    ]
    s = text.split(",")
    if len(s) == 2:
        date = s[0]
        time = s[1]
    else:
        print(text)
        date = text[:-5].strip()
        time = text[-5:].strip()
    return {"date" : date, "time" : time}
                

launch_dict = initiate_launch_dict()
for x in list_data:
    if len(x) != 11:
        print(x[0], len(x), x)
    else:
        launch_dict['Flight No.'] += [x[10]]
        launch_dict['Launch site'] += [x[3]]
        launch_dict['Payload'] += [x[4]]
        launch_dict['Payload mass'] += [get_mass(x[5], True)]
        launch_dict['Orbit'] += [x[6]]
        launch_dict['Customer'] += [x[7]]
        launch_dict['Launch outcome'] += [x[8]]
        launch_dict['Version Booster'] += [x[2]]
        launch_dict['Booster landing'] += [x[9]]
        launch_dict['Date'] += [split_date_time(x[1])["date"]]
        launch_dict['Time'] += [split_date_time(x[1])["time"]]
        # launch_dict['Date'] += [date_time(x[1], True)]
        # launch_dict['Time'] += [date_time(x[1], True)]

df = pd.DataFrame(launch_dict)
df

8 7 [8, 'Orbcomm-OG2', '172kg (379lb)', 'LEO', 'Orbcomm', 'Partial failure', '4']
6 4 [6, 'B1023.2 (side) ♺', 'Success(ground pad)', 'FH 1']
7 4 [7, 'B1025.2 (side) ♺', 'Success(ground pad)', 'FH 1']
8 4 [8, 'B1052.1(side)', 'Success(ground pad)', 'FH 2']
9 4 [9, 'B1053.1(side)', 'Success(ground pad)', 'FH 2']
18 4 [18, 'B1052.2(side) ♺', 'Success(ground pad)', 'FH 3']
19 4 [19, 'B1053.2(side) ♺', 'Success(ground pad)', 'FH 3']
18 August 2020 14:31
18 August 2020 14:31
30 August 202023:18
30 August 202023:18
3 September 202012:46:14
3 September 202012:46:14
6 October 202011:29:34
6 October 202011:29:34
18 October 202012:25:57
18 October 202012:25:57
24 October 202015:31:34
24 October 202015:31:34
5 November 202023:24:23
5 November 202023:24:23
16 November 202000:27
16 November 202000:27
21 November 202017:17:08
21 November 202017:17:08
25 November 202002:13
25 November 202002:13
6 December 202016:17:08
6 December 202016:17:08
13 December 202017:30:00
13 December 202017:30:00
19 Decembe

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,"CCAFS,SLC-40",Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,F9 v1.0B0003.1,Failure(parachute),4 June 2010,18:45
1,2,"CCAFS,SLC-40",Dragon demo flight C1(Dragon C101),0,LEO (ISS),NASA (COTS)NRO,Success,F9 v1.0B0004.1,Failure(parachute),8 December 2010,15:43
2,3,"CCAFS,SLC-40",Dragon demo flight C2+(Dragon C102),525kg,LEO (ISS),NASA (COTS),Success,F9 v1.0B0005.1,Not attempted,22 May 2012,07:44
3,4,"CCAFS,SLC-40",SpaceX CRS-1(Dragon C103),"4,700kg",LEO (ISS),NASA (CRS),Success,F9 v1.0B0006.1,No attempt,8 October 2012,00:35
4,5,"CCAFS,SLC-40",SpaceX CRS-2(Dragon C104),"4,877kg",LEO (ISS),NASA (CRS),Success,F9 v1.0B0007.1,Not attempted,1 March 2013,15:10
...,...,...,...,...,...,...,...,...,...,...,...
120,117,"CCSFS,SLC-40",Starlink 27 v1.0 (60 satellites),"15,600kg",LEO,SpaceX,Success,F9 B5 ♺B1051.10,Success(drone ship),9 May 2021,06:42
121,118,"KSC,LC-39A",Starlink 26 v1.0 (52 Satellites) Capella-6 &Ty...,"~14,000kg",LEO,SpaceX Capella Space and Tyvak,Success,F9 B5 ♺B1058.8,Success(drone ship),15 May 2021,22:56
122,119,"CCSFS,SLC-40",Starlink 28 v1.0 (60 Satellites),"15,600kg",LEO,SpaceX,Success,F9 B5 ♺B1063.2,Success(drone ship),26 May 2021,18:59
123,120,"KSC,LC-39A",SpaceX CRS-22(Dragon C209.1),"3,328kg",LEO (ISS),NASA (CRS),Success,F9 B5 B1067.1,Success(drone ship),3 June 2021,17:29


In [17]:
df.to_csv("spacex_webscraped.csv", index=False, sep=";")

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
